In [ ]:
import pickle
import numpy as np

In [ ]:
with open(f'results/outputs_judge.p', 'rb') as f:
    outputs_0 = pickle.load(f)

In [ ]:
outputs_0

In [ ]:
with open(f'results/outputs_neo_0.p', 'rb') as f:
    outputs_0 = pickle.load(f)
with open(f'results/outputs_neo_1.p', 'rb') as f:
    outputs_1 = pickle.load(f)
with open(f'results/outputs_neo_2.p', 'rb') as f:
    outputs_2 = pickle.load(f)
with open(f'results/outputs_neo_test.p', 'rb') as f:
    outputs_test = pickle.load(f)

## Improved extraction

In [ ]:
isInterpretation = lambda x: 'interpr' in x
isRational = lambda x: 'ration' in x
isEmotional = lambda x: 'emotion' in x
isAgreement = lambda x: 'agree' in x and not 'disagree' in x
isDisagreement = lambda x: 'disagree' in x
isAny = lambda x: isInterpretation(x) or isRational(x) or isEmotional(x) or isAgreement(x) or isDisagreement(x)

In [ ]:
def enhance_outputs(outputs):
    for cur_output in outputs:
        other_text = cur_output['other_text']
        for t in other_text:
            if isInterpretation(t):
                cur_output['interpretation'] += 1
            if isRational(t):
                cur_output['rational'] += 1
            if isEmotional(t):
                cur_output['emotional'] += 1
            if isAgreement(t):
                cur_output['agreement'] += 1
            if isDisagreement(t):
                cur_output['disagreement'] += 1
        cur_output['other_text'] = list(filter(lambda x: not isAny(x), other_text))
        cur_output['other'] = len(cur_output['other_text'])
                
        

In [ ]:
enhance_outputs(outputs_0)
enhance_outputs(outputs_1)
enhance_outputs(outputs_2)
enhance_outputs(outputs_test)

In [ ]:
def predict_probas(res):
    #res = self.predict_claim(claim, n_prompts=n_prompts, n_samples=n_samples)
    #self.all_res.append(res)
    classes_text = ['interpretation', 'rational', 'emotional', 'agreement', 'disagreement'] 
    probas = np.zeros(len(classes_text))
    for i, cls in enumerate(classes_text):
        probas[i] = res[cls]
    probas /= np.sum(probas)
    return probas

def predict(outputs):
    probas = np.array(list(map(lambda x: predict_probas(x), outputs)))
    return probas

In [ ]:
preds_0 = predict(outputs_0)
preds_1 = predict(outputs_1)
preds_2 = predict(outputs_2)
preds_test = predict(outputs_test)

In [ ]:
np.savez('results/results_neo_0_enh.npz', preds=preds_0)
np.savez('results/results_neo_1_enh.npz', preds=preds_1)
np.savez('results/results_neo_2_enh.npz', preds=preds_2)
np.savez('results/results_neo_test_enh.npz', preds=preds_test)